<a href="https://colab.research.google.com/github/ElizabethWaithera/EDA/blob/main/chat_with_Gemini_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

! pip install -q --upgrade google-generativeai langchain-google-genai python-dotenv
!pip install langchain
!pip install pypdf
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

##Install necessary libraries

In [ ]:
import pathlib
import textwrap

from IPython.display import display,Markdown

# Used to securely store your API key
from google.colab import userdata

In [ ]:
import google.generativeai as genai
#retrieve the key
GOOGLE_API_KEY=userdata.get('Googler')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.3, google_api_key=GOOGLE_API_KEY)

In [ ]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd

from langchain import PromptTemplate


# Document Loader
from langchain.document_loaders import PyPDFLoader

# Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Vector DataBase
from langchain.vectorstores import Chroma, FAISS, Pinecone

# Chains - sequence of calls
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain

In [ ]:
import pathlib as p

# Define the path to the data folder
data_folder = p.Path.cwd() / "data"

# Create the data folder if it doesn't exist
data_folder.mkdir(parents=True, exist_ok=True)

# Path to the local PDF file
local_pdf_file = "/content/MCCR_1229_18.pdf"

# Destination path for the PDF file within the data folder
pdf_file = data_folder / local_pdf_file.split("/")[-1]

# Copy the local PDF file to the data folder
p.Path(local_pdf_file).replace(pdf_file)

PosixPath('/content/data/MCCR_1229_18.pdf')

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# Convert the PosixPath object to a string
pdf_file_path = str(pdf_file)

# Initialize the PyPDFLoader with the file path
pdf_loader = PyPDFLoader(pdf_file_path)

# Now you can proceed with loading and splitting the PDF
pages = pdf_loader.load_and_split()

In [ ]:
print(f"Number of pages: {len(pages)}")

Number of pages: 3


In [ ]:
# Initialize the RecursiveCharacterTextSplitter with specified parameters
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50000, chunk_overlap= 0)

# Concatenate page contents into a single string with double line breaks between each page's content
context = "\n\n".join(str(p.page_content) for p in pages)

# Split the concatenated text into chunks using the initialized text splitter
texts = text_splitter.split_text(context)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain.vectorstores import Chroma, FAISS, Pinecone


embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

# Convert the texts into vectors using the provided embeddings
vector_index = Chroma.from_texts(texts, embeddings)

# Convert the vector index into a retriever
vector_index = vector_index.as_retriever()

In [ ]:
prompt_template = """
  Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
  provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
  Context:\n {context}?\n
  Question: \n{question}\n

  Answer:
"""

prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])

In [ ]:
question = "What was the Scheduled Case Activity"

chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
docs = vector_index.get_relevant_documents(question)
docs


[Document(page_content="WILDLIFE CRIME WATCH COURT REPORT  \n \n \nAttendance  Date : 18th Jan 2024 \nCourt Location:  Kibera  Court 2 \nCase File # : MCCR/ 1229 /18 \nScheduled Case Activity : Defence Hearing    \nCourt Monitor name/initials:  IM \n \nThe matter was presented before Hon. S.O. Temu  (SPM) .  \nProsecutor : Mr. Ngetich  \nBoth the accused, Meshack Mutua and Benard Mulee, and their representative Mr. Kibet \nwere present in court.   \nWitness 1 : Meshack Mutua Musioka Occupation : businessman who deals in fruits.  \nOn 30th Sep 2018, his day began in Eldoret where he operates his business. He woke up  at 5 \nam and went to the Soko Mjinga market in his car, KCD 124N a Toyota Wish, where he \nbought 7 sacks of oranges. He then went to Kapsabet market to set up shop. He got to \nKapsabet before 7 and went about his business as usual. By noon, he had been able to sell 5 \nsacks of orange s leaving only 2. He had been planning to go home to Machakos on that day \nand the rem

In [ ]:
response = chain(
    {"input_documents":docs, "question": question}
    , return_only_outputs=True)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [ ]:
response

{'output_text': 'Defence Hearing'}

In [ ]:
response

{'output_text': 'Defence Hearing'}